In [68]:
%run code/import_libs.py
%run code/functions.py
#%run code/functions_v1.py
%run code/Build_Forms.py
%run code/initial_data_prep.py

%matplotlib inline

initial_pred = df.drop(columns=["price"]).copy()
initial_price = df[["price"]]

pred_fin, price_fin = transform_data(initial_pred, initial_price)

# Create OLS linear model

pred_int = sm.add_constant(pred_fin)
model = sm.OLS(price_fin,pred_int).fit()
mean_price_2014_2015=initial_price.mean()[0]


coef_df=model.params.reset_index()
coef_df.columns=["Column","Value"]




In [69]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.881
Model:                            OLS   Adj. R-squared:                  0.880
Method:                 Least Squares   F-statistic:                     768.1
Date:                Thu, 31 Mar 2022   Prob (F-statistic):               0.00
Time:                        22:55:31   Log-Likelihood:                 6068.2
No. Observations:               21143   AIC:                        -1.173e+04
Df Residuals:                   20940   BIC:                        -1.011e+04
Df Model:                         202                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             9.0093      0.189     47.794      0.000       8.640       9.379
sqft_living       0.4568      0.005     85.821      0.000       0.446       0.467
sqft_lot          0.0758      0.002     34.604      0.000       0.071       0.080
gra_4            -0.6419      0.187     -3.430      0.001      -1.009      -0.275
gra_5            -0.6524      0.184     -3.542      0.000      -1.013      -0.291
gra_6            -0.5540      0.184     -3.013      0.003      -0.914      -0.194
gra_7            -0.4493      0.184     -2.443      0.015      -0.810      -0.089
gra_8            -0.3479      0.184     -1.890      0.059      -0.709       0.013
gra_9            -0.2269      0.184     -1.232      0.218      -0.588       0.134
gra_10           -0.1227      0.184     -0.666      0.505      -0.484       0.238
gra_11            0.0060      0.184      0.032      0.974      -0.356       0.368
gra_12            0.1388      0.185      0.749      0.454      -0.224       0.502
gra_13            0.4334      0.191      2.265      0.024       0.058       0.808
zip_98002         0.0046      0.016      0.279      0.780      -0.027       0.037
zip_98003         0.0414      0.015      2.801      0.005       0.012       0.070
zip_98004         1.1624      0.015     79.952      0.000       1.134       1.191
zip_98005         0.7891      0.017     45.252      0.000       0.755       0.823
zip_98006         0.6952      0.013     53.156      0.000       0.670       0.721
zip_98007         0.7035      0.019     37.964      0.000       0.667       0.740
zip_98008         0.7046      0.015     47.150      0.000       0.675       0.734
zip_98010         0.2373      0.021     11.277      0.000       0.196       0.279
zip_98011         0.4751      0.016     28.878      0.000       0.443       0.507
zip_98014         0.2797      0.019     14.387      0.000       0.242       0.318
zip_98019         0.3134      0.017     18.863      0.000       0.281       0.346
zip_98022         0.0467      0.016      2.965      0.003       0.016       0.078
zip_98023         0.0035      0.013      0.274      0.784      -0.022       0.029
zip_98024         0.4007      0.023     17.437      0.000       0.356       0.446
zip_98027         0.5351      0.013     39.732      0.000       0.509       0.561
zip_98028         0.4314      0.015     29.240      0.000       0.402       0.460
zip_98029         0.6311      0.015     43.490      0.000       0.603       0.660
zip_98030         0.0588      0.015      3.878      0.000       0.029       0.089
zip_98031         0.1081      0.015      7.233      0.000       0.079       0.137
zip_98032         0.0034      0.019      0.177      0.860      -0.034       0.041
zip_98033         0.8154      0.013     61.358      0.000       0.789       0.841
zip_98034         0.5763      0.013     45.461   

In [70]:
coef_df[coef_df['Column'] == "wat_YES"]

,Column,Value
82,wat_YES,0.526085


In [71]:



def calculate_price (sqft_living, sqft_lot, basement, yr_built, zipcode, grade, waterfront, \
                     view, mean_price, coef_df=coef_df, output='yes'): #output='yes'):
    if view == '-':
        view='NONE'

    if waterfront == 'YES': waterfront='WATERFRONT'
    else: waterfront = 'NONE'

    b0,b1,b2,b3,b4,b5,b6,b7,b8 = get_coeff( yr_built, zipcode, grade, waterfront, view, coef_df)
    y=round( np.exp(b0 + b1*np.log(sqft_living) + b2*np.log(sqft_lot)  + b3 + b4 + b5 + b6 + b7 +b7+b8) )

    y=y*(mean_price/mean_price_2014_2015)
    if output == 'yes': print('{:,.0f}'.format(y))
    return y,b0,b1,b2,b3,b4,b5,b6,b7,b8



abs_test_results=[]
test_results=[]
for i in range(100):
    X=np.array(df[['sqft_living','sqft_lot', 'basement', 'yr_built','zipcode','grade','waterfront','view', 'price']].sample())[0]
    sqft_living, sqft_lot, basement, yr_built, zipcode, grade, waterfront, view ,price=X

#    b0,b1,b2,b3,b4,b5,b6,b7 = get_coeff( year=year, zipcode=zipcode, grade=grade, water=water, view=view, coef_df=coef_df)
#    y=round( np.exp(b0 + b1*np.log(sqft_living) + b2 + b3 + b4 + b5 + b6 + b7*np.log(sqft_lot)) )

    y,b0,b1,b2,b3,b4,b5,b6,b7,b8= calculate_price (sqft_living, sqft_lot, basement, yr_built, zipcode, \
                                                   grade, waterfront, view, mean_price, coef_df=coef_df, output='no')

    miss_abs= np.abs(100-(y/price*100))
    miss= 100-(y/price*100)
    test_results.append(miss)
    abs_test_results.append(miss_abs)

avg_miss=sum(test_results)/len(test_results)
avg_miss_abs=sum(abs_test_results)/len(abs_test_results)
print (avg_miss, avg_miss_abs)


-0.27705302125594344 13.430633793367416


In [72]:
#print (X)
print(y,b0,b1,b2,b3,b4,b5,b6,b7,b8)
print (len(test_results))

636665.6488796031 9.009306472364264 0.4567834012190797 0.07578454862818891 -0.020399110360911354 -0.12505837566371925 0.5763376353723886 -0.34786102632076954 0 0.011270778581614516
100


In [73]:
title=form_items[0].children[0]
meanW=form_items[1].children[0]
zipW=form_items[2].children[1]
yearW=form_items[2].children[2]
gradeW=form_items[3].children[1]
livingW=form_items[4].children[1]
basementW=form_items[4].children[2]
lotW=form_items[5].children[1]
viewW=form_items[6].children[1]
waterW=form_items[6].children[2]

#buttonW=form_items[5].children[0]

output = widgets.interactive_output(calculate_price, { 'view':viewW,'waterfront':waterW,
    'zipcode': zipW, 'yr_built':yearW, 'grade':gradeW, 'basement':basementW, "mean_price":meanW,
    'sqft_living':livingW,'sqft_lot':lotW, } )



ui = widgets.VBox([form, output])

output.layout={'border': '3px solid green', 'width':'150px'}

#predictor=display(ui)


In [74]:
display(ui)